This Python script simulates the dynamics of charged particles under relativistic conditions using the Pygame library. It is designed to visualize and explore concepts such as electromagnetic fields, synchrotron radiation, and the effects of relativity on particle interactions. Here's a breakdown of what the script does:

### Imports and Constants
- **Pygame** is used for graphical rendering and interaction.
- **Numpy** is utilized for numerical operations, especially in vector and matrix computations.
- Constants like `WIDTH`, `HEIGHT`, `SPEED_OF_LIGHT`, and `ELEMENTARY_CHARGE` are defined for use in simulations.

### Synchrotron Function
- `synchrotron_function(x)`: Approximates a function related to synchrotron radiation, which particles emit when they are accelerated in magnetic fields.

### Particle Class
- Represents charged particles with properties like position (`x`, `y`, `z`), `charge`, `mass`, and `velocity`.
- `update`: Updates particle's state based on electromagnetic fields and other particles. It calculates forces from given electric and magnetic fields and updates the particle's velocity and position considering relativistic effects (like mass increase at speeds close to the speed of light and time dilation).
- `synchrotron_radiation_spectrum` and `synchrotron_radiation_power`: Calculate the spectrum and power of synchrotron radiation emitted by the particle.
- `retarded_position` and `retarded_fields`: Compute the position and electromagnetic fields of the particle at previous times, accounting for the finite speed of light (this is part of what makes the simulation 'relativistic').
- `relativistic_mass` and `time_dilation`: Compute the relativistic mass and time dilation effects based on the particle’s velocity.

### ParticleManager Class
- Manages a collection of `Particle` instances.
- `add_particle`: Adds a new particle to the simulation.
- `draw`: Draws all particles on the Pygame screen.
- `update_interactions`: Updates interactions between all particles, considering retarded fields, which simulate the delay in the effect of one particle's field on another due to the finite speed of light.
- `update`: Updates all particles’ states and interactions between them.

### Field Functions
- `electric_field` and `magnetic_field`: Functions to compute the electric and magnetic fields at any point in space based on predefined distributions or configurations.

### Simulation Loop
- Initializes Pygame and sets up the simulation environment.
- Enters a loop where it processes user events (like quitting or adding particles with mouse clicks), updates the state of the simulation (particle movements and interactions), and renders the updated state to the screen.
- Uses Pygame’s drawing functions to visualize particles and possibly field lines or intensity.

### Key Concepts Illustrated by the Script
- **Relativistic Effects**: The script accounts for effects such as time dilation and relativistic mass increase, essential when simulating particles moving at speeds close to the speed of light.
- **Electromagnetic Interactions**: It computes interactions between particles based on electromagnetic fields, including those generated by the particles themselves, considering the retardation effect due to the finite speed of light.
- **Synchrotron Radiation**: It calculates and possibly visualizes the radiation emitted by charged particles when accelerated, which is significant in many high-energy physics contexts.

Overall, this script provides a platform for visualizing and understanding the complex dynamics involved in relativistic particle interactions, including electromagnetic forces and radiation. It serves as an educational tool for illustrating concepts from electromagnetism and relativistic physics.

In [ ]:
# V 1.6 Reletivistic particle 
# Reletivistic columnb law 
# electromagnetic particle interactions
# 
import pygame
import random
import numpy as np
from pygame import surfarray

# Constants
WIDTH, HEIGHT = 800, 600
BLACK = (0, 0, 0)
RED = (255, 255, 255)
SPEED_OF_LIGHT = 299792458  # Speed of light in m/s
 
ELEMENTARY_CHARGE = 1.602176634e-19  # Elementary charge in Coulombs
EPSILON_0 = 8.8541878128e-12  # Permittivity of free space in F/m
 
def synchrotron_function(x):
    # Synchrotron function approximation
    if x < 0.1:
        return 2.14 * x ** (1/3)
    elif x < 5:
        return 1.25 * x ** 0.29 * np.exp(-x)
    else:
        return 1.25 * x ** 0.29 * np.exp(-x) * (1 + 0.78 * x ** (-2/3))
class Particle:
    def __init__(self, x, y, z, charge, mass):
        self.x, self.y, self.z = x, y, z
        self.charge, self.mass = charge, mass
        self.radius = 5  # The radius of the particle
        self.color = RED  # Color of the particle
        self.velocity = np.array([0.0, 0.0, 0.0])  # Initialize velocity 

    def update(self, electric_field_function, magnetic_field_function, particles):
        electric_field = electric_field_function(self.x, self.y, self.z) 
        magnetic_field = magnetic_field_function(self.x, self.y, self.z)
        
        # Use the synchrotron_power as needed, e.g., for energy loss calculations
        synchrotron_power = self.synchrotron_radiation_power(magnetic_field)
        frequency = 1e15  # Example frequency (in Hz) for spectrum calculation
        # Use the synchrotron_spectrum as needed, e.g., for radiation emission calculations
        synchrotron_spectrum = self.synchrotron_radiation_spectrum(magnetic_field, frequency)
        
        # Calculate the retarded fields from all other particles
        retarded_electric_field = np.zeros(3)
        retarded_magnetic_field = np.zeros(3)
        for particle in particles:
            if particle != self:
                E, B = particle.retarded_fields(self.x, self.y, self.z, 0)  # Assuming instantaneous fields
                retarded_electric_field += E
                retarded_magnetic_field += B

        # Calculate the total electric and magnetic fields
        total_electric_field = electric_field + retarded_electric_field
        total_magnetic_field = magnetic_field + retarded_magnetic_field

        # Calculate the electric force as a vector
        electric_force = self.charge * total_electric_field
        # Calculate cross product velocity, total_magnetic_field
        magnetic_force = self.charge * np.cross(self.velocity, total_magnetic_field)
        
     
        #  total force
        total_force = electric_force + magnetic_force  
        
        # Calculate acceleration as a vector
        acceleration = total_force / self.relativistic_mass()
        # Update velocity based on acceleration
        self.velocity = np.clip(self.velocity + acceleration, -SPEED_OF_LIGHT, SPEED_OF_LIGHT)
        # Update position based on velocity
        self.x += self.velocity[0] * self.time_dilation()
        self.y += self.velocity[1] * self.time_dilation()
        self.z += self.velocity[2] * self.time_dilation()
        # Wrap around screen, 
        self.x %= WIDTH
        self.y %= HEIGHT 
        
 
        
    def synchrotron_radiation_spectrum(self, magnetic_field, frequency):
        # Calculate the synchrotron radiation spectrum at a given frequency
        speed = np.linalg.norm(self.velocity)
        if speed >= SPEED_OF_LIGHT:
            # Rescale the velocity to ensure it's below the speed of light.
            self.velocity *= (SPEED_OF_LIGHT * 0.999) / speed
            speed = SPEED_OF_LIGHT * 0.999  # Update speed to match the rescaled velocity.
        B = np.linalg.norm(magnetic_field)
        
        v_perp = np.linalg.norm(np.cross(self.velocity, magnetic_field)) / B
        gamma = 1 / np.sqrt(1 - (np.linalg.norm(self.velocity) / SPEED_OF_LIGHT) ** 2)
        critical_frequency = (3 * ELEMENTARY_CHARGE * B * gamma ** 2) / (2 * np.pi * self.mass)
        x = frequency / critical_frequency
        spectrum = (np.sqrt(3) * ELEMENTARY_CHARGE ** 2 * B * gamma) / (2 * np.pi ** 2 * EPSILON_0 * SPEED_OF_LIGHT) * x * synchrotron_function(x)
        return spectrum
        
        
    def synchrotron_radiation_power(self, magnetic_field):
        # Calculate the synchrotron radiation power emitted by the particle
        speed = np.linalg.norm(self.velocity)
        if speed >= SPEED_OF_LIGHT:
            # Rescale the velocity to ensure it's below the speed of light.
            self.velocity *= (SPEED_OF_LIGHT * 0.999) / speed
            speed = SPEED_OF_LIGHT * 0.999  # Update speed to match the rescaled velocity.
        B = np.linalg.norm(magnetic_field)
        v_perp = np.linalg.norm(np.cross(self.velocity, magnetic_field)) / B
        gamma = 1 / np.sqrt(1 - (np.linalg.norm(self.velocity) / SPEED_OF_LIGHT) ** 2)
        P = (2 * ELEMENTARY_CHARGE ** 2 * v_perp ** 2 * gamma ** 4) / (3 * EPSILON_0 * SPEED_OF_LIGHT ** 3 * self.mass ** 2)
        
        return P

    def retarded_position(self, t):
        # Calculate the retarded position of the particle at time t
        retarded_velocity = self.velocity / (1 + np.dot(self.velocity, self.velocity) / SPEED_OF_LIGHT**2)
        retarded_position = np.array([self.x, self.y, self.z]) - self.velocity * t
        return retarded_position

    def retarded_fields(self, x, y, z, t):
        # Calculate the retarded electric and magnetic fields at position (x, y, z) and time t
        r = np.array([x, y, z]) - self.retarded_position(t)
        r_mag = np.linalg.norm(r)
        r_unit = r / r_mag
        retarded_velocity = self.velocity / (1 + np.dot(self.velocity, self.velocity) / SPEED_OF_LIGHT**2)
        
        # Calculate the retarded electric field
        E = (self.charge / (4 * np.pi * EPSILON_0)) * (
            (r_unit / r_mag**2) * (1 - np.dot(retarded_velocity, retarded_velocity) / SPEED_OF_LIGHT**2) +
            (np.cross(r_unit, np.cross(r_unit, retarded_velocity)) / (r_mag * SPEED_OF_LIGHT**2))
        )
        
        # Calculate the retarded magnetic field
        B = np.cross(r_unit, E) / SPEED_OF_LIGHT
        
        return E, B

    def relativistic_mass(self):
        # Calculate the relativistic mass of the particle
        speed = np.linalg.norm(self.velocity)
        
        if speed >= SPEED_OF_LIGHT:
            speed = SPEED_OF_LIGHT * 0.999  # Set speed to slightly less than the speed of light
        lorentz_factor = 1 / np.sqrt(1 - (speed / SPEED_OF_LIGHT) ** 2)
        return self.mass * lorentz_factor

    def time_dilation(self):
        # Calculate the time dilation factor
        speed = np.linalg.norm(self.velocity)
        if speed >= SPEED_OF_LIGHT:
            speed = SPEED_OF_LIGHT * 0.999  # Set speed to slightly less than the speed of light
        lorentz_factor = 1 / np.sqrt(1 - (speed / SPEED_OF_LIGHT) ** 2)
        return 1 / lorentz_factor

    def draw(self, screen):
        # Draw the particle on the given screen
        pygame.draw.circle(screen, self.color, (int(self.x), int(self.y)), self.radius)
        
        
class ParticleManager:
    def __init__(self):
        self.particles = []  # List to hold all particles

    def add_particle(self, x, y, z, charge, mass):
        # Add a new particle to the list
        self.particles.append(Particle(x, y, z, charge, mass))

    def draw(self, screen):
        # Draw all particles
        for particle in self.particles:
            particle.draw(screen)
            
    def update_interactions(self):
        c = SPEED_OF_LIGHT
        for i, particle1 in enumerate(self.particles):
            for j, particle2 in enumerate(self.particles):
                if i >= j:  # Avoid double counting and self-interaction
                    continue

                # Calculate the time delay between particles
                # This is a simplification; in reality, you'd need to solve for t such that
                # the position of particle2 at time (current_time - t) matches the retarded position
                # relative to particle1, which involves iterative or analytical solutions.
                distance = np.linalg.norm([particle2.x - particle1.x, particle2.y - particle1.y, particle2.z - particle1.z])
                time_delay = distance / c

                # Get the retarded electric and magnetic fields from particle2 at the position of particle1,
                # considering the time delay
                E_retarded, B_retarded = particle2.retarded_fields(particle1.x, particle1.y, particle1.z, time_delay)

                # Now apply forces based on these fields instead of the instant Coulomb's law
                # For example, using just the electric field part for simplicity (add magnetic if needed):
                force = particle1.charge * E_retarded

                # Update velocity of particle1 based on this force; remember to consider relativistic mass
                particle1.velocity += force / particle1.relativistic_mass()

                # For particle2, do the same but with the fields it would feel from particle1
                # (This is simplified and assumes reciprocity, which may need adjustment for moving charges)
                E_retarded, B_retarded = particle1.retarded_fields(particle2.x, particle2.y, particle2.z, time_delay)
                force = particle2.charge * E_retarded
                particle2.velocity += force / particle2.relativistic_mass()

    def update(self, electric_field, magnetic_field):
        for particle in self.particles:
            particle.update(electric_field, magnetic_field, self.particles)
        self.update_interactions() 


############################################################################# 

def electric_field(x, y, z):
    # _chaos  Constants
    k = 8.9875517873681764e9
    Q = 1
    source_x, source_y, source_z = 400, 300, 0  # Position of the center of the distribution
    sigma_x, sigma_y, sigma_z = 100, 100, 50  # Standard deviations of the distribution
    Lx, Ly = 800, 600  # Dimensions of the simulation domain
    epsilon = 1e-9  # Small value to avoid division by zero

    # Compute differences in coordinates
    dx, dy, dz = x - source_x, y - source_y, z - source_z

    # Compute the Gaussian distribution
    gaussian = Q * np.exp(-0.5 * ((dx/sigma_x)**2 + (dy/sigma_y)**2 + (dz/sigma_z)**2))

    # Compute the chaotic perturbation
    perturbation = 0.1 * (np.sin(2 * np.pi * 2 * x / Lx - z)
                          + 0.5 * np.cos(3 * np.pi * 6 * y / Ly - 2*z)
                          + 0.2 * np.sin(x*y*z))

    # Combine the Gaussian distribution and chaotic perturbation
    field_magnitude = gaussian #* (1 + 2*perturbation)
    #field_magnitude =   perturbation

    # Compute the electric field components
    field_x = k * field_magnitude * dx / (sigma_x**2 )
    field_y = k * field_magnitude * dy / (sigma_y**2 )
    field_z = k * field_magnitude * dz / (sigma_z**2 )

    return np.array([field_x, field_y, field_z])

def magnetic_field(x, y, z):
    # Example: a uniform magnetic field in the z direction
    Bz = 3.5
    By = 0
    Bx = 0     
    return np.array([Bx, By, Bz ]) # Bx, By, Bz
#############################################################################

# Initialize Pygame
pygame.init()
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Particle Simulation")
clock = pygame.time.Clock()

# Particle manager to manage multiple particles
particle_manager = ParticleManager()
 
particle_manager.add_particle(400, 300, 0, -1.0, 2e7)  #
 
x_grid, y_grid, z_grid = np.meshgrid(range(WIDTH), range(HEIGHT), np.array(0))

electric_fields = electric_field(x_grid, y_grid, z_grid)
magnetic_fields = magnetic_field(x_grid, y_grid, z_grid)

normalized_electric_field =electric_fields# (electric_fields / np.max(np.abs(electric_fields))) * 25500

field_surface = pygame.Surface((WIDTH, HEIGHT))

surfarray.blit_array(field_surface, normalized_electric_field[0,:,:,0].reshape(600, 800).T)

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.MOUSEBUTTONDOWN:  # Check for mouse click
            # Get mouse position and add a new particle there
            x, y = pygame.mouse.get_pos()
            charge = 1# random.choice([-1.0, 1.0])  # Randomly choose charge
            particle_manager.add_particle(x, y, 2e5, charge, 1.0)  # Add new particle at mouse position          
    
    screen.fill(BLACK) 
    screen.blit(field_surface, (0, 0))     
    
    particle_manager.update(electric_field, magnetic_field)
         
    particle_manager.draw(screen)
    pygame.display.flip()
    clock.tick(60)
